In [1]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, year, count, when, desc, sum, to_timestamp, row_number, regexp_replace, expr, asc
from pyspark.sql.types import DecimalType
from pyspark.sql import functions as F

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3693,application_1732639283265_3639,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
### QUERY 2 (a)
APP_NAME = "Closed Cases"
spark = SparkSession.builder.appName(APP_NAME).getOrCreate() # reconstructing because we dont need 4 spark executors now

# crime data
d1 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv",
    header=True, inferSchema=True).select(col('DATE OCC'), col('AREA NAME'), col('Status'), col('LAT'), col('LON'))
d2 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",
    header=True, inferSchema=True).select(col('DATE OCC'), col('AREA NAME'), col('Status'), col('LAT'), col('LON'))
crime_data = d1.union(d2)

# DATAFRAME API BEGIN #
start = time.time()
crime_data = crime_data.withColumn('YEAR', year(to_timestamp(col('DATE OCC'), 'MM/dd/yyyy hh:mm:ss a'))).withColumn('closed', when(~col('Status').isin('IC', 'UKN'), 1).otherwise(0))
crime_data_agg = crime_data.groupBy('YEAR', 'AREA NAME').agg(
    count('*').alias("total_cases"),
    sum('closed').alias('closed_cases')
)
crime_cc_rates = crime_data_agg.withColumn('closed_case_rate', when(col('total_cases') > 0, col('closed_cases')/col('total_cases') * 100).otherwise(None)).drop('closed_cases').drop('total_cases')
window_spec = Window.partitionBy('year').orderBy(desc('closed_case_rate'))
ranked_df = crime_cc_rates.withColumn("#", row_number().over(window_spec))
top3_df = ranked_df.filter(col("#") <= 3)
# top3_df.explain(True)
top3_df_collect = top3_df.collect()
end = time.time()
dataframe_time = end-start
print("DATAFRAME performance:", end-start)
print("DATAFRAME results:")
top3_df.show(n=top3_df.count(), truncate=False)
# DATAFRAME API END #

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DATAFRAME performance: 13.183066368103027
DATAFRAME results:
+----+-----------+------------------+---+
|YEAR|AREA NAME  |closed_case_rate  |#  |
+----+-----------+------------------+---+
|2010|Rampart    |32.84713448949121 |1  |
|2010|Olympic    |31.515289821999087|2  |
|2010|Harbor     |29.36028339237341 |3  |
|2011|Olympic    |35.040060090135206|1  |
|2011|Rampart    |32.4964471814306  |2  |
|2011|Harbor     |28.51336246316431 |3  |
|2012|Olympic    |34.29708533302119 |1  |
|2012|Rampart    |32.46000463714352 |2  |
|2012|Harbor     |29.509585848956675|3  |
|2013|Olympic    |33.58217940999398 |1  |
|2013|Rampart    |32.1060382916053  |2  |
|2013|Harbor     |29.723638951488557|3  |
|2014|Van Nuys   |32.0215235281705  |1  |
|2014|West Valley|31.49754809505847 |2  |
|2014|Mission    |31.224939855653567|3  |
|2015|Van Nuys   |32.265140677157845|1  |
|2015|Mission    |30.463762673676303|2  |
|2015|Foothill   |30.353001803658852|3  |
|2016|Van Nuys   |32.194518462124094|1  |
|2016|West Vall

In [3]:
# crime data
d1 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv",
    header=True, inferSchema=True).select(col('DATE OCC'), col('AREA NAME'), col('Status'), col('LAT'), col('LON'))
d2 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",
    header=True, inferSchema=True).select(col('DATE OCC'), col('AREA NAME'), col('Status'), col('LAT'), col('LON'))
crime_data = d1.union(d2)

query = """
WITH processed_data AS (
    SELECT 
        YEAR(TO_TIMESTAMP(`DATE OCC`, 'MM/dd/yyyy hh:mm:ss a')) AS YEAR,
        `AREA NAME`,
        CASE WHEN `Status` NOT IN ('IC', 'UKN') THEN 1 ELSE 0 END AS closed
    FROM crime_data
),
aggregated_data AS (
    SELECT
        YEAR,
        `AREA NAME`,
        COUNT(*) AS total_cases,
        SUM(closed) AS closed_cases
    FROM processed_data
    GROUP BY YEAR, `AREA NAME`
),
crime_cc_rates AS (
    SELECT
        YEAR,
        `AREA NAME`,
        (CASE WHEN total_cases > 0 THEN (closed_cases / total_cases) * 100 ELSE NULL END) AS closed_case_rate
    FROM aggregated_data
),
ranked_data AS (
    SELECT
        YEAR,
        `AREA NAME`,
        closed_case_rate,
        ROW_NUMBER() OVER (PARTITION BY YEAR ORDER BY closed_case_rate DESC) AS rank
    FROM crime_cc_rates
)
SELECT
    YEAR,
    `AREA NAME`,
    closed_case_rate,
    rank AS `#`
FROM ranked_data
WHERE rank <= 3
"""

# SQL API BEGIN #
start = time.time()
crime_data.createOrReplaceTempView("crime_data") # register the crime_data DataFrame as a temporary view
top3_sql = spark.sql(query)
top3_sql_collect = top3_sql.collect()
end = time.time()
sql_time = end-start
print("SQL performance:", end-start)
print("SQL results:")
top3_sql.show(n=top3_df.count(), truncate=False)
# SQL API END #

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SQL performance: 8.888805389404297
SQL results:
+----+-----------+------------------+---+
|YEAR|AREA NAME  |closed_case_rate  |#  |
+----+-----------+------------------+---+
|2010|Rampart    |32.84713448949121 |1  |
|2010|Olympic    |31.515289821999087|2  |
|2010|Harbor     |29.36028339237341 |3  |
|2011|Olympic    |35.040060090135206|1  |
|2011|Rampart    |32.4964471814306  |2  |
|2011|Harbor     |28.51336246316431 |3  |
|2012|Olympic    |34.29708533302119 |1  |
|2012|Rampart    |32.46000463714352 |2  |
|2012|Harbor     |29.509585848956675|3  |
|2013|Olympic    |33.58217940999398 |1  |
|2013|Rampart    |32.1060382916053  |2  |
|2013|Harbor     |29.723638951488557|3  |
|2014|Van Nuys   |32.0215235281705  |1  |
|2014|West Valley|31.49754809505847 |2  |
|2014|Mission    |31.224939855653567|3  |
|2015|Van Nuys   |32.265140677157845|1  |
|2015|Mission    |30.463762673676303|2  |
|2015|Foothill   |30.353001803658852|3  |
|2016|Van Nuys   |32.194518462124094|1  |
|2016|West Valley|31.4014643

In [4]:
### QUERY 2 (b)
crime_data.write.mode("overwrite").parquet("s3://groups-bucket-dblab-905418150721/group33/CrimeData.parquet")
crime_data_parquet = spark.read.parquet("s3://groups-bucket-dblab-905418150721/group33/CrimeData.parquet")

def process_data_dataframe(crime_data):
    crime_data = crime_data.withColumn('YEAR', year(to_timestamp(col('DATE OCC'), 'MM/dd/yyyy hh:mm:ss a'))) \
                           .withColumn('closed', when(~col('Status').isin('IC', 'UKN'), 1).otherwise(0))
    crime_data_agg = crime_data.groupBy('YEAR', 'AREA NAME').agg(
        count('*').alias("total_cases"),
        sum('closed').alias('closed_cases')
    )
    crime_cc_rates = crime_data_agg.withColumn(
        'closed_case_rate', 
        when(col('total_cases') > 0, col('closed_cases') / col('total_cases') * 100).otherwise(None)
    ).drop('closed_cases').drop('total_cases')
    window_spec = Window.partitionBy('YEAR').orderBy(desc('closed_case_rate'))
    ranked_df = crime_cc_rates.withColumn("#", row_number().over(window_spec))
    top3_df = ranked_df.filter(col("#") <= 3)
    return top3_df

# DATAFRAME API BEGIN #
start = time.time()
top3_df_parquet = process_data_dataframe(crime_data_parquet)
top3_df_parquet_collect = top3_df_parquet.collect()
end = time.time()
parquet_time = end-start
print("DATAFRAME performance:", end-start)
print("DATAFRAME results:")
top3_df_parquet.show(n=top3_df_parquet.count(), truncate=False)
# DATAFRAME API END #

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DATAFRAME performance: 4.5808703899383545
DATAFRAME results:
+----+-----------+------------------+---+
|YEAR|AREA NAME  |closed_case_rate  |#  |
+----+-----------+------------------+---+
|2010|Rampart    |32.84713448949121 |1  |
|2010|Olympic    |31.515289821999087|2  |
|2010|Harbor     |29.36028339237341 |3  |
|2011|Olympic    |35.040060090135206|1  |
|2011|Rampart    |32.4964471814306  |2  |
|2011|Harbor     |28.51336246316431 |3  |
|2012|Olympic    |34.29708533302119 |1  |
|2012|Rampart    |32.46000463714352 |2  |
|2012|Harbor     |29.509585848956675|3  |
|2013|Olympic    |33.58217940999398 |1  |
|2013|Rampart    |32.1060382916053  |2  |
|2013|Harbor     |29.723638951488557|3  |
|2014|Van Nuys   |32.0215235281705  |1  |
|2014|West Valley|31.49754809505847 |2  |
|2014|Mission    |31.224939855653567|3  |
|2015|Van Nuys   |32.265140677157845|1  |
|2015|Mission    |30.463762673676303|2  |
|2015|Foothill   |30.353001803658852|3  |
|2016|Van Nuys   |32.194518462124094|1  |
|2016|West Vall

In [6]:
print("SQL Speedup:", dataframe_time/sql_time)
print("Parquet Speedup:", dataframe_time/parquet_time)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SQL Speedup: 1.4831088982798082
Parquet Speedup: 2.8778518591268054